In [1]:

from google.colab import drive
drive.mount('/content/drive')



In [2]:
!pip install unidecode
!pip install gensim==4.3.1
!pip install fasttext
!pip install tensorflow

In [3]:
import gensim
import re
import logging
import numpy as np
import pandas as pd
import multiprocessing

from re import sub
from time import time 
from unidecode import unidecode
from gensim.models import Word2Vec
from collections import defaultdict
from gensim.models import KeyedVectors
from gensim.test.utils import get_tmpfile
from gensim.models.phrases import Phrases, Phraser

import tensorflow as tf
import keras
from keras.models import Sequential
from keras import layers
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.preprocessing.text import Tokenizer
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.utils import pad_sequences, to_categorical
from sklearn.model_selection import train_test_split


2023-05-24 22:08:53.659341: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-24 22:08:53.661366: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-24 22:08:53.691048: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-24 22:08:53.691678: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-24 22:08:54.365338: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [4]:
#data loading
colnames=['comment'] 
hiphop = pd.read_csv("hiphopheads.csv", on_bad_lines='skip',engine='python', header = None, names = colnames)
jazz = pd.read_csv("jazz.csv", on_bad_lines='skip',engine='python', header = None, names = colnames)
punk = pd.read_csv("punk.csv", on_bad_lines='skip',engine='python', header = None, names = colnames)
metal = pd.read_csv("metal.csv", on_bad_lines='skip',engine='python', header = None, names = colnames)
classical = pd.read_csv("classicalmusic.csv", on_bad_lines='skip',engine='python', header = None, names = colnames)

In [5]:

# training BERT takes significantly more time for larger datasets.
# We train BERT on fewer rows.

fileList2 = []
#fileList2.append(hiphop.head(30000))
fileList2.append(jazz.sample(2000))
#fileList2.append(punk.head(10000))
fileList2.append(metal.sample(2000))
#fileList2.append(classical.head(250))

In [6]:
merge = pd.concat(fileList2)
# We remove lines longer than 500
merge = merge[merge['comment'].str.len()<500]

fileList = []
fileList.append(merge)

print(type(merge))
print(merge)

<class 'pandas.core.frame.DataFrame'>
                                                       comment
t1_jdzmrpo   Breathless\r -Kenny G\n\nI really hate quite a...
t1_j66jw2y   I just discovered them. I cannot stop listenin...
t1_j118cx5   That is one of the several reasons I posted th...
t1_j3oqeev                                        Oh heck yes!
t1_jcs5vay                      Sam Wilkes - Live on the Green
...                                                        ...
t1_j0x2fxt                                        Fantastic!!!
t1_i9he4n9   More of the latter but has some heavier elemen...
t1_i3ke1tl   I’m typically not into Tech Death, but my old ...
*I am a bot   and this action was performed automatically. ...
t1_j90e412   I saw Revocation with support from Goatwhore, ...

[3036 rows x 1 columns]


In [7]:
def text_to_word_list(text, remove_polish_letters):
    ''' Pre process and convert texts to a list of words 
    method inspired by method from eliorc github repo: https://github.com/eliorc/Medium/blob/master/MaLSTM.ipynb'''
    text = remove_polish_letters(text)
    text = str(text)
    text = text.lower()

    # Clean the text
    text = sub(r"[^A-Za-z0-9^,!?.\/'+]", " ", text)
    text = sub(r"\+", "", text)
    text = sub(r",", "", text)
    text = sub(r"\.", "", text)
    text = sub(r"!", "", text)
    text = sub(r"\?", "", text)
    text = sub(r"'", "", text)
    text = sub(r":", "", text)
    text = sub(r"\s{2,}", " ", text)

    text = text.split()

    return text  

In [8]:
from sklearn.cluster import KMeans
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

In [9]:
def create_tfidf_dictionary(x, transformed_file, features):
    '''
    create dictionary for each input sentence x, where each word has assigned its tfidf score
    
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer

    '''
    vector_coo = transformed_file[x.name].tocoo()
    vector_coo.col = features.iloc[vector_coo.col].values
    dict_from_coo = dict(zip(vector_coo.col, vector_coo.data))
    return dict_from_coo

def replace_tfidf_words(x, transformed_file, features):
    '''
    replacing each word with it's calculated tfidf dictionary with scores of each word
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer
    '''
    dictionary = create_tfidf_dictionary(x, transformed_file, features)   
    return list(map(lambda y:dictionary[f'{y}'], x.title.split()))

In [10]:
def replace_sentiment_words(word, sentiment_dict):
    '''
    replacing each word with its associated sentiment score from sentiment dict
    '''
    try:
        out = sentiment_dict[word]
    except KeyError:
        out = 0
    return out

In [11]:
predictedList = []

In [12]:
def divide_to_parts(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]


for file_ in fileList:
  parts = divide_to_parts(file_.comment, 3000)    
  file_['rate'] = 1
  file_cleaned = file_.dropna().drop_duplicates().reset_index(drop=True).rename(columns={'comment':'title'})
  file_cleaned = file_cleaned[file_cleaned.rate!=0]
  file_cleaned.rate.value_counts()/len(file_cleaned)

  file_cleaned.title = file_cleaned.title.apply(lambda x: text_to_word_list(x, unidecode))
  file_model = file_cleaned.copy()
  file_model = file_model[file_model.title.str.len()>1]


  parts = divide_to_parts(file_model.title, 3000)
  for part in parts:    
    #Detect bigrams with gensim's Phraser module.
    sent = [row for row in part]
    phrases = Phrases(sent, min_count=1, progress_per=50000)
    bigram = Phraser(phrases)
    sentences = bigram[sent]

    #word embeddings with CBOW word2vec algorithm found in gensim module
    w2v_model = Word2Vec(min_count=3,
                      window=4,
                      vector_size=300,
                      sample=1e-5, 
                      alpha=0.03, 
                      min_alpha=0.0007, 
                      negative=20,
                      workers=multiprocessing.cpu_count()-1)

    start = time()

    w2v_model.build_vocab(sentences, progress_per=50000)

    print('Time to build vocab: {} mins'.format(round((time() - start) / 60, 2)))

    start = time()

    w2v_model.train(sentences,
                    total_examples=w2v_model.corpus_count,
                    epochs=30,
                    report_delay=1)

    print('Time to train the model: {} mins'.format(round((time() - start) / 60, 2)))

    w2v_model.init_sims(replace=True)


    w2v_model.save("word2vec.model")
    
    file_export = file_model.copy()
    file_export['old_title'] = file_export.title
    file_export.old_title = file_export.old_title.str.join(' ')
    file_export.title = file_export.title.apply(lambda x: ' '.join(bigram[x]))
    file_export.rate = file_export.rate.astype('int8')

    file_export[['title', 'rate']].to_csv('cleaned_dataset.csv', index=False)

    # KMeans clusters for sentiment


    word_vectors = Word2Vec.load("word2vec.model").wv

    model = KMeans(n_clusters=2,
                  max_iter=1000,
                  random_state=42,
                  n_init=50)
    model.fit(X=word_vectors.vectors.astype('double'))

    word_vectors.similar_by_vector(model.cluster_centers_[1], topn=10, restrict_vocab=None)

    positive_cluster_index = 1
    positive_cluster_center = model.cluster_centers_[positive_cluster_index]
    negative_cluster_center = model.cluster_centers_[1-positive_cluster_index]

    words = pd.DataFrame(word_vectors.index_to_key)
    words.columns = ['words']
    words['vectors'] = words.words.apply(lambda x: word_vectors[f'{x}'])
    words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
    words.cluster = words.cluster.apply(lambda x: x[0])
    words['cluster_value'] = [1 if i==positive_cluster_index else -1 for i in words.cluster]
    words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
    words['sentiment_coeff'] = words.closeness_score * words.cluster_value

    words[['words', 'sentiment_coeff']].to_csv('sentiment_dictionary.csv', index=False)

    #Labelling data

    final_file = pd.read_csv('cleaned_dataset.csv')
    sentiment_map = pd.read_csv('sentiment_dictionary.csv')
    sentiment_dict = dict(zip(sentiment_map.words.values, sentiment_map.sentiment_coeff.values))

    file_weighting = final_file.copy()
    tfidf = TfidfVectorizer(tokenizer=lambda y: y.split(), norm=None)
    tfidf.fit(file_weighting.title)
    features = pd.Series(tfidf.get_feature_names_out())
    transformed = tfidf.transform(file_weighting.title)

    replaced_tfidf_scores = file_weighting.apply(lambda x: replace_tfidf_words(x, transformed, features), axis=1)

    replaced_closeness_scores = file_weighting.title.apply(lambda x: list(map(lambda y: replace_sentiment_words(y, sentiment_dict), x.split())))

    replacement_df = pd.DataFrame(data=[replaced_closeness_scores, replaced_tfidf_scores, file_weighting.title, file_weighting.rate]).T
    replacement_df.columns = ['sentiment_coeff', 'tfidf_scores', 'sentence', 'sentiment']
    replacement_df['sentiment_rate'] = replacement_df.apply(lambda x: np.array(x.loc['sentiment_coeff']) @ np.array(x.loc['tfidf_scores']), axis=1)
    replacement_df['prediction'] = (replacement_df.sentiment_rate>0).astype('int8')
    replacement_df['sentiment'] = [1 if i==1 else 0 for i in replacement_df.sentiment]

    predictedList.append(replacement_df)




Time to build vocab: 0.0 mins
Time to train the model: 0.05 mins


/tmp/ipykernel_550078/2054725627.py:51: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)
/home/piotrzab/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [13]:
merge = pd.concat(predictedList)

tempList = []
tempList.append(merge)

In [14]:
listTwoElems = []

for elem in tempList:
  listTwoElems.append(elem[['sentence', 'prediction']])


In [15]:
print(listTwoElems[0])

                                               sentence  prediction
0     breathless kenny_g i really_hate quite_a few o...           1
1     i just_discovered them i cannot stop_listening...           0
2     that is one_of the several reasons i posted th...           0
3                                           oh heck yes           1
4                          sam_wilkes live on the green           0
...                                                 ...         ...
2732  black_sabbath iron_maiden slayer slipknot and ...           1
2733                  songs about life going in circles           0
2734  more of the_latter but has some heavier elemen...           1
2735  im typically not into_tech death but my old ro...           1
2736  i_saw revocation with support from goatwhore a...           0

[2737 rows x 2 columns]


In [16]:
with pd.option_context('display.max_colwidth', None):
  display(listTwoElems[0])

,sentence,prediction
0,breathless kenny_g i really_hate quite_a few of his albums but this_one is special,1
1,i just_discovered them i cannot stop_listening my_fav musician is sergio mendes so i recognize many of the songs,0
2,that is one_of the several reasons i posted this someone told_me that there_are some british documentaries on_youtube that are superior to this series i_suppose it is on me to_explore which_ones are best,0
3,oh heck yes,1
4,sam_wilkes live on the green,0
...,...,...
2732,black_sabbath iron_maiden slayer slipknot and some motley crue,1
2733,songs about life going in circles,0
2734,more of the_latter but has some heavier elements i much prefer beyond hypothermia/uyhs era cave in but i like this new_album quite_a bit its super long though around 80 minutes i_think,1
2735,im typically not into_tech death but my old room mate got_me into revocation which then led me to archspire and_inferi the new archspire album is absolutely nuts i really love how theres still an_actual song amongst_the crazy technicalities with that said_they end_up having_a lot_of similar sounding songs but when i listen_to archspire its for exactly that style cant_wait to_see them and_inferi in may,1


BERT MODEL

In [17]:
!pip install transformers==4.28.0
!pip install -U datasets

In [18]:
import transformers
transformers.__version__

'4.28.0'

In [19]:
data = listTwoElems[0]
data.rename(columns={'prediction':'label'}, inplace=True)
data.rename(columns={'sentence':'text'}, inplace=True)
data.sample(5)

/tmp/ipykernel_550078/25399030.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={'prediction':'label'}, inplace=True)
/tmp/ipykernel_550078/25399030.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={'sentence':'text'}, inplace=True)


,text,label
644,pat_metheny bright_size life on ecm records,0
2433,crypt of ice is that album i know is good but ...,0
2646,oh_yeah love this guys truly rednecks,1
270,pretty_sure thats the_90s remake,1
288,speaking of herbie yesterday i heard a version...,1


In [20]:
from datasets import Dataset

dataset_ = Dataset.from_pandas(data)
dataset = dataset_.train_test_split(0.1)

In [21]:
model_checkpoint = 'distilbert-base-uncased'
batch_size = 128

In [22]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [23]:
def process(x):
  return tokenizer(x['text'])

train_ds = dataset['train'].map(process)
test_ds = dataset['test'].map(process)

Map:   0%|          | 0/2463 [00:00<?, ? examples/s]

Map:   0%|          | 0/274 [00:00<?, ? examples/s]

In [24]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier

In [25]:
!pip install --upgrade accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [26]:
args = TrainingArguments(
    f'{model_checkpoint}_sentiment_analysis',
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    learning_rate = 2e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = 5,
    weight_decay = 0.01,
    load_best_model_at_end = True,
    metric_for_best_model = 'accuracy'
)

In [27]:
from datasets import load_metric
import numpy as np

metric = load_metric('glue', 'sst2')

def compute_metrics(eval_preds):
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

/tmp/ipykernel_550078/2777850806.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', 'sst2')


In [28]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [29]:
trainer.evaluate([train_ds[0]])

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.6170973777770996,
 'eval_accuracy': 1.0,
 'eval_runtime': 0.066,
 'eval_samples_per_second': 15.155,
 'eval_steps_per_second': 15.155}

In [30]:
trainer.train()

/home/piotrzab/anaconda3/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.692657,0.521898
2,No log,0.691940,0.551095
3,No log,0.688147,0.562044
4,No log,0.684854,0.580292
5,No log,0.681816,0.580292


TrainOutput(global_step=100, training_loss=0.6762546539306641, metrics={'train_runtime': 1079.321, 'train_samples_per_second': 11.41, 'train_steps_per_second': 0.093, 'total_flos': 361669115180544.0, 'train_loss': 0.6762546539306641, 'epoch': 5.0})

We can see the accuracy is around 55%